In [71]:
import os
import sys
import pandas as pd
sys.path.append(os.path.abspath('../Scripts'))
import warnings
warnings.filterwarnings('ignore')

In [72]:
import eda as ed

In [73]:
df = 'data.csv'
data = ed.load_data(df)

2024-10-10 08:30:23,845 - INFO - Attempting to load data from data.csv
2024-10-10 08:30:24,620 - INFO - Successfully loaded data from data.csv


In [74]:
data['TransactionStartTime'] = pd.to_datetime(data['TransactionStartTime'])


In [75]:
current_date = pd.to_datetime('now')

In [76]:
import pytz

In [77]:
print(data['TransactionStartTime'].dtype)

datetime64[ns, UTC]


In [78]:
data['TransactionStartTime'] = data['TransactionStartTime'].dt.tz_convert(None)

In [79]:

# Feature Engineering

# 1. Transaction Frequency
transaction_frequency = data.groupby('CustomerId')['TransactionId'].count().reset_index()
transaction_frequency.columns = ['CustomerId', 'Transaction_Frequency']

# 2. Average Transaction Amount
avg_transaction_amount = data.groupby('CustomerId')['Amount'].mean().reset_index()
avg_transaction_amount.columns = ['CustomerId', 'Avg_Transaction_Amount']

# 3. Total Transaction Volume
total_transaction_volume = data.groupby('CustomerId')['Amount'].sum().reset_index()
total_transaction_volume.columns = ['CustomerId', 'Total_Transaction_Volume']

#4. Transaction Timing
# Calculate Recency
current_date = pd.to_datetime('now')
#data['Recency'] = (current_date - data['TransactionStartTime']).dt.days

# Convert current_date to EAT (UTC+3)
# Localize current_date to EAT (UTC+3)
current_date_eat = current_date.tz_localize('Africa/Addis_Ababa')

# Assuming data['TransactionStartTime'] is in UTC, first localize it to UTC
#data['TransactionStartTime'] = data['TransactionStartTime'].dt.tz_localize('UTC')

# Now convert TransactionStartTime to EAT
data['TransactionStartTime_EAT'] = data['TransactionStartTime'].dt.tz_convert('Africa/Addis_Ababa')

# Calculate Recency in days
data['Recency'] = (current_date_eat - data['TransactionStartTime_EAT']).dt.days

recency = data.groupby('CustomerId')['Recency'].min().reset_index()
recency.columns = ['CustomerId', 'Transaction_Recency']

# 5. Variability of Transaction Amounts
transaction_variability = data.groupby('CustomerId')['Amount'].std().reset_index()
transaction_variability.columns = ['CustomerId', 'Transaction_Amount_Variability']


#6. Extracting Time-Based Features from TransactionStartTime
data['TransactionHour'] = data['TransactionStartTime'].dt.hour
data['TransactionDay'] = data['TransactionStartTime'].dt.day
data['TransactionMonth'] = data['TransactionStartTime'].dt.month
data['TransactionYear'] = data['TransactionStartTime'].dt.year

# 7. Calculate the mean values for new time-based features for each customer
transaction_hour = data.groupby('CustomerId')['TransactionHour'].mean().reset_index()
transaction_hour.columns = ['CustomerId', 'Avg_Transaction_Hour']

transaction_day = data.groupby('CustomerId')['TransactionDay'].mean().reset_index()
transaction_day.columns = ['CustomerId', 'Avg_Transaction_Day']

transaction_month = data.groupby('CustomerId')['TransactionMonth'].mean().reset_index()
transaction_month.columns = ['CustomerId', 'Avg_Transaction_Month']

transaction_year = data.groupby('CustomerId')['TransactionYear'].mean().reset_index()
transaction_year.columns = ['CustomerId', 'Avg_Transaction_Year']

# Merge all features into a single DataFrame
features = transaction_frequency.merge(avg_transaction_amount, on='CustomerId') \
                                .merge(total_transaction_volume, on='CustomerId') \
                                .merge(recency, on='CustomerId') \
                                .merge(transaction_variability, on='CustomerId')\
                                .merge(transaction_hour, on='CustomerId') \
                                .merge(transaction_day, on='CustomerId') \
                                .merge(transaction_month, on='CustomerId') \
                                .merge(transaction_year, on='CustomerId')

# Reset index if necessary
features.reset_index(drop=True, inplace=True)

# Output the features DataFrame
print(features.head())


TypeError: Cannot convert tz-naive timestamps, use tz_localize to localize

In [54]:
features

,CustomerId,Transaction_Frequency,Avg_Transaction_Amount,Total_Transaction_Volume,Transaction_Recency,Transaction_Amount_Variability,Avg_Transaction_Hour,Avg_Transaction_Day,Avg_Transaction_Month,Avg_Transaction_Year
0,CustomerId_1,1,-10000.000000,-10000.0,2149,NaN,16.000000,21.000000,11.000000,2018.000000
1,CustomerId_10,1,-10000.000000,-10000.0,2149,NaN,16.000000,21.000000,11.000000,2018.000000
2,CustomerId_1001,5,4000.000000,20000.0,2154,6558.963333,7.800000,16.000000,11.000000,2018.000000
3,CustomerId_1002,11,384.090909,4225.0,2091,560.498966,13.454545,14.727273,8.636364,2018.272727
4,CustomerId_1003,6,3333.333333,20000.0,2077,6030.478146,14.333333,1.000000,2.000000,2019.000000
...,...,...,...,...,...,...,...,...,...,...
3737,CustomerId_992,6,3333.333333,20000.0,2070,6088.240030,10.000000,8.000000,2.000000,2019.000000
3738,CustomerId_993,5,4000.000000,20000.0,2091,6745.368782,15.000000,18.000000,1.000000,2019.000000
3739,CustomerId_994,101,5384.881188,543873.0,2066,14800.656784,13.900990,18.475248,5.207921,2018.623762
3740,CustomerId_996,17,8176.470588,139000.0,2133,4433.329648,14.470588,7.000000,12.000000,2018.000000


In [68]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder  # For scaling and encoding

In [67]:
features.describe(include= 'object')

,CustomerId
count,95662
unique,3742
top,CustomerId_7343
freq,4091


In [66]:
features.dtypes

CustomerId                         object
Transaction_Frequency               int64
Avg_Transaction_Amount            float64
Total_Transaction_Volume          float64
Transaction_Recency                 int64
Transaction_Amount_Variability    float64
TransactionHour                     int32
TransactionDay                      int32
TransactionMonth                    int32
TransactionYear                     int32
dtype: object

## Encoding catagorical varibles

In [70]:
# Assuming 'features' is your DataFrame after feature engineering

# One-Hot Encoding for nominal categorical variables 
features = pd.get_dummies(features, columns=['CustomerId'], drop_first=True)

# Label Encoding for ordinal categorical variables if any (not shown in the example)
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
# Example: if 'SomeOrdinalCategory' is an ordinal variable
# features['SomeOrdinalCategory'] = label_encoder.fit_transform(features['SomeOrdinalCategory'])


KeyError: "None of [Index(['CustomerId'], dtype='object')] are in the [columns]"

In [ ]:
# Check for missing values
missing_values = features.isnull().sum()

# Option 1: Imputation
# Filling missing values with mean, median, or mode for numerical features
for column in features.select_dtypes(include=['float64', 'int64']).columns:
    features[column].fillna(features[column].mean(), inplace=True)  # or use median or mode as required

# Option 2: Removal
# features.dropna(inplace=True)  # Remove rows with missing values if few
